# ETL

# Fetch data and data preparation

I chose data coming from credit card applications and records. The data is downloaded from Kaggle and can be found at https://www.kaggle.com/rikdifos/credit-card-approval-prediction

## <font>Download files</font>

<font color="red"> You must first install the Kaggle API CLI by running 'pip install kaggle' and also have the Kaggle credentials (kaggle.json) in the same folder of this NB</font>

In [1]:
# set up the Kaggle credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

!pwd
# download the zip file if not found 
![ -f data/credit-card-approval-prediction.zip ] || pip install kaggle && cd data && kaggle datasets download rikdifos/credit-card-approval-prediction

# unzip the datasets
!unzip -o data/credit-card-approval-prediction.zip -d data

kaggle.json
/Users/arosero/afroserom/coursera/IBM_advanced_DS
credit-card-approval-prediction.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  data/credit-card-approval-prediction.zip
  inflating: data/application_record.csv  
  inflating: data/credit_record.csv  


## Settings

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import helpers.settings as sts
sts.print_settings(sts)

BEST_ESTIMATOR_FILENAME : best_estimator_0.0.1.pkl
DATASET_TRAIN_FILENAME : dataset_train.parquet
DATASET_VALIDATION_FILENAME : dataset_validation.parquet
ETL_VERSION : 0.0.1
MODEL_FILENAME : model.pkl
MODEL_VERSION : 0.0.1
PREPROCESSOR_FILENAME : preprocessor_0.0.1.pkl
TRAINED_BEST_ESTIMATOR_FILENAME : trained_best_estimator_0.0.1.pkl
color : <class 'helpers.settings.color'>
print_settings : <function print_settings at 0x7f8f70abbf70>



## Imports

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    classification_report,
    plot_precision_recall_curve,
    precision_recall_curve,
    average_precision_score,
)
from xgboost import XGBClassifier
from category_encoders.woe import WOEEncoder
from sklearn.preprocessing import RobustScaler
import numpy as np
import seaborn as sns

## Format conversion

Convert to parquet format for more flexible usage

In [6]:
pd.read_csv("data/application_record.csv").to_parquet("data/application.parquet", index=False)
pd.read_csv("data/credit_record.csv").to_parquet("data/credit_record.parquet", index=False)

In [7]:
application = pd.read_parquet("data/application.parquet")
credit_record = pd.read_parquet("data/credit_record.parquet")

## Data sample

In [8]:
application.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [9]:
credit_record.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


## Creating label

### Month balance & status

Month balance: 

The month of the extracted data is the starting point, backwards, 0 is the current month, -1 is the previous month, and so on

Status:

0: 1-29 days past due 1: 30-59 days past due 2: 60-89 days overdue 3: 90-119 days overdue 4: 120-149 days overdue 5: Overdue or bad debts, write-offs for more than 150 days C: paid off that month X: No loan for the month

<font color="red">I will assume that any overdue of 60 or more days corresponds to a default</font>

In [10]:
defaulted_user_ids = credit_record.query("STATUS not in ['0','1','C','X']")["ID"].unique()

In [11]:
credit_record_unique = credit_record.drop_duplicates(subset=["ID"],keep="last",ignore_index=True).copy()

In [12]:
credit_record_unique["LABEL"] = credit_record["ID"].apply(lambda x: 1 if x in defaulted_user_ids else 0)

Proportion of good and bad customers

In [13]:
credit_record_unique["LABEL"].value_counts(normalize=True)

0    0.988583
1    0.011417
Name: LABEL, dtype: float64

## Duplicates

In [14]:
len(application.ID.unique())

438510

In [15]:
application_duplicates = application[application.duplicated(subset=[col for col in application.columns if col != "ID"], keep=False)]
application_duplicates

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
5,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438550,6840100,F,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,None,1.0
438551,6840102,F,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,None,1.0
438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,None,2.0


In [16]:
application_no_duplicates_first = application[~application.duplicated(subset=[col for col in application.columns if col != "ID"], keep="first")]
application_no_duplicates_first

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
7,5008812,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,None,1.0
10,5008815,M,Y,Y,0,270000.0,Working,Higher education,Married,House / apartment,-16872,-769,1,1,1,1,Accountants,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438541,6837707,M,N,Y,0,202500.0,Working,Higher education,Civil marriage,House / apartment,-13510,-2309,1,1,0,0,Laborers,2.0
438545,6839651,F,N,Y,3,99000.0,Pensioner,Secondary / secondary special,Single / not married,House / apartment,-18832,365243,1,0,0,0,None,1.0
438547,6839917,F,N,Y,0,180000.0,Pensioner,Higher education,Married,House / apartment,-10966,-2704,1,0,0,0,None,2.0
438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,None,1.0


In [17]:
len(application_no_duplicates_first.ID.unique())

90085

In [18]:
len(credit_record["ID"].unique())

45985

In [19]:
len(set(application_no_duplicates_first.ID).intersection(set(credit_record.ID)))

9709

In [20]:
duplicate_id_groups = pd.DataFrame(application.groupby(by=[col for col in application.columns if col != "ID"])["ID"].apply(lambda x: list(np.unique(x))).reset_index(drop=True))

In [21]:
duplicate_id_groups

,ID
0,"[6231976, 6231977]"
1,"[5126174, 5126175, 5995345]"
2,"[6618535, 6690062, 6690063]"
3,[6722801]
4,"[5771303, 5771305]"
...,...
62603,"[5135534, 5135535, 5135536, 5135537, 5135538, ..."
62604,"[5949365, 5949366, 5949368, 5949369, 5949370, ..."
62605,"[6272384, 6471231, 6471232, 6471233, 6471234, ..."
62606,"[5342630, 5342633, 5342634, 5342635, 5342637, ..."


In [22]:
pd.options.display.max_rows = 1000

In [23]:
credit_record.query(f'ID in [5135534, 5135535, 5135536, 5135537, 5135538]').sort_values(["ID","MONTHS_BALANCE"], ascending=False)

,ID,MONTHS_BALANCE,STATUS
945201,5135538,0,X
945202,5135538,-1,X
945203,5135538,-2,X
945204,5135538,-3,X
945205,5135538,-4,X
945206,5135538,-5,X
945207,5135538,-6,X
945208,5135538,-7,X
945209,5135538,-8,X
945210,5135538,-9,X


In [24]:
credit_record.query(f'ID in [5061207, 5061210, 5061211]').sort_values(["ID","MONTHS_BALANCE"], ascending=False)

,ID,MONTHS_BALANCE,STATUS
505528,5061211,0,0
505529,5061211,-1,0
505530,5061211,-2,0
505531,5061211,-3,0
505532,5061211,-4,1
505533,5061211,-5,1
505534,5061211,-6,0
505535,5061211,-7,0
505536,5061211,-8,0
505537,5061211,-9,0


### Merge applications with credit records

In [25]:
df = application.merge(credit_record_unique[["ID","LABEL"]], how="inner", on="ID")

### Drop duplicates

In [26]:
df.drop_duplicates(subset=["ID"], keep="first", inplace=True)

In [27]:
df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,LABEL
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0,0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0,0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0


## Split datasets

In [28]:
X = df.iloc[:,1:-1] 
y = df.iloc[:,-1] 

In [29]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=42)

In [30]:
y_train.value_counts(normalize=True)

0    0.986963
1    0.013037
Name: LABEL, dtype: float64

## Dump datasets

In [31]:
pd.concat([X_train, y_train], axis=1).to_parquet(f"data/{sts.DATASET_TRAIN_FILENAME}", index=False)
pd.concat([X_test, y_test], axis=1).to_parquet(f"data/{sts.DATASET_VALIDATION_FILENAME}", index=False)